In [1]:
# Importa as bibliotecas
# Validar o mínimo de CPF
# Criar uma home direcionando para o crud e para o gráfico

import os
from dotenv import load_dotenv

import pandas as pd
import psycopg2 as pg
import sqlalchemy
from sqlalchemy import create_engine
import panel as pn
import matplotlib.pyplot as plt

In [2]:
# Carrega as variáveis do arquivo .env

load_dotenv()

True

In [3]:
# Lê as variáveis de ambiente

DB_HOST = os.getenv('DB_HOST')
DB_NAME = os.getenv('DB_NAME')
DB_USER = os.getenv('DB_USER')
DB_PASS = os.getenv('DB_PASS')

In [4]:
# Cria conexão com psycopg2 usando as variáveis carregadas

con = pg.connect(host=DB_HOST, dbname=DB_NAME, user=DB_USER, password=DB_PASS)

In [5]:
# Define a string de conexão para o SQLAlchemy, utilizando as variáveis do .env
# Cria o objeto engine do SQLAlchemy que será usado para conectar e executar comandos no banco

cnx = f'postgresql://{DB_USER}:{DB_PASS}@{DB_HOST}/{DB_NAME}'

engine = create_engine(cnx)

In [6]:
# Executa a consulta SQL para buscar todos os 
# registros da tabela 'pessoa' no banco PostgreSQL 
# e carrega o resultado em um DataFrame do pandas


query = "select * from usuario;" 
df = pd.read_sql_query(query, cnx)

df.head()

,id_usuario,nome_completo,email,senha,pais,estado,cidade,rua,num_residencia
0,1,João Silva,joao1@email.com,123,Brasil,SP,São Paulo,Rua A,10.0
1,2,Maria Souza,maria2@email.com,123,Brasil,RJ,Rio de Janeiro,Rua B,20.0
2,3,Carlos Lima,carlos3@email.com,123,Brasil,MG,Belo Horizonte,Rua C,30.0
3,4,Ana Rocha,ana4@email.com,123,Brasil,RS,Porto Alegre,Rua D,40.0
4,5,Pedro Alves,pedro5@email.com,123,Brasil,SC,Florianópolis,Rua E,50.0


In [ ]:
# Inicializa as extensões do Panel necessárias:
# - Tabulator para tabelas interativas
# - Notificações na interface

pn.extension()
pn.extension('tabulator')
pn.extension(notifications=True)

In [ ]:
# Criação da Home sem conectar os cliques ainda
btn_go_user_crud = pn.widgets.Button(name="📋 CRUD Usuário", button_type="primary", width=250)
btn_go_donation_crud = pn.widgets.Button(name="📋 CRUD Doação", button_type="primary", width=250)
btn_go_campaign_crud = pn.widgets.Button(name="📋 CRUD Camapanha", button_type="primary", width=250)
btn_go_cpoint_crud = pn.widgets.Button(name="📋 CRUD Ponto de Coleta", button_type="primary", width=250)
btn_go_graficos = pn.widgets.Button(name="📊 Ir para Gráficos", button_type="success", width=250)

In [ ]:
# Container principal da aplicação
main_area = pn.Column()

def create_home_view():
    home = pn.Column(
        pn.pane.Markdown("## 🏠 Tela Inicial"),
        pn.pane.Markdown("Escolha uma opção abaixo:"),
        pn.Spacer(height=20),
        btn_go_user_crud,
        btn_go_donation_crud,
        btn_go_campaign_crud,
        btn_go_cpoint_crud,
        btn_go_graficos,
        align="center",
        sizing_mode="stretch_both",
        margin=(50,50,50,50)
    )
    return home


In [ ]:
# Funções de navegação
def go_to_home():
    main_area.clear()
    main_area.append(create_home_view())

In [ ]:
def create_btn_voltar():
    btn_voltar_home = pn.widgets.Button(name='⬅ Voltar para Home', button_type='default')
    btn_voltar_home.on_click(lambda e: go_to_home())
    return btn_voltar_home

In [ ]:
# Variável auxiliar para consultas sem filtro
flag=''

# Widgets de entrada de dados
def create_form_widgets():
    return {
        "id": pn.widgets.IntInput(name="ID Usuário", value=0),
        "nome": pn.widgets.TextInput(name="Nome Completo"),
        "email": pn.widgets.TextInput(name="Email"),
        "senha": pn.widgets.PasswordInput(name="Senha"),
        "cidade": pn.widgets.TextInput(name="Cidade"),
        "estado": pn.widgets.TextInput(name="Estado", placeholder="Ex: CE")
    }

def get_campaign_widgets():
    return {
        "id": pn.widgets.IntInput(name="ID Campanha", value=0),
        "nome": pn.widgets.TextInput(name="Nome da Campanha"),
        "data_inicio": pn.widgets.DatePicker(name="Data Início"),
        "data_fim": pn.widgets.DatePicker(name="Data Fim"),
        "id_inst": pn.widgets.IntInput(name="ID Instituição Responsável")
    }

def get_cpoint_widgets():
    return {
        "id": pn.widgets.IntInput(name="ID Ponto", value=0),
        "nome": pn.widgets.TextInput(name="Nome do Local"),
        "cidade": pn.widgets.TextInput(name="Cidade"),
        "rua": pn.widgets.TextInput(name="Rua")
    }

def get_donation_widgets():
    return {
        "id": pn.widgets.IntInput(name="ID Doação", value=0),
        "data": pn.widgets.DatePicker(name="Data da Doação"),
        "desc": pn.widgets.TextInput(name="Descrição/Item"),
        "id_cont": pn.widgets.IntInput(name="ID Contribuidor"),
        "id_camp": pn.widgets.IntInput(name="ID Campanha"),
        "id_ponto": pn.widgets.IntInput(name="ID Ponto Coleta")
    }

In [ ]:
# Botões de ações CRUD
buttonConsultar = pn.widgets.Button(name='Consultar', button_type='primary')
buttonInserir = pn.widgets.Button(name='Inserir', button_type='success')
buttonExcluir = pn.widgets.Button(name='Excluir', button_type='danger')
buttonAtualizar = pn.widgets.Button(name='Atualizar', button_type='warning')

# Botão para voltar à Home (na tela CRUD)
buttonVoltar = pn.widgets.Button(name='⬅ Voltar para Home', button_type='default')


In [ ]:
def queryAll():
    """
    Consulta todos os registros da tabela 'usuario' e retorna um Tabulator.
    """
    df = pd.read_sql_query("SELECT * FROM usuario", cnx)
    return pn.widgets.Tabulator(df, show_index=False, sizing_mode='stretch_width', height=400)

def executar_sql(query, params=None):
    try:
        cursor = con.cursor()
        cursor.execute(query, params)
        con.commit()
        cursor.close()
        pn.state.notifications.success("Operação realizada com sucesso!")
        return True
    except Exception as e:
        con.rollback()
        pn.state.notifications.error(f"Erro: {str(e)}")
        return False

In [ ]:
def on_consultar_user(w):
    """Consulta pelo CPF ou retorna todos."""
    try:
        nome_completo = w["nome"].value
        query = f"SELECT * FROM usuario WHERE ('{nome_completo}'='{flag}' OR nome_completo='{nome_completo}')"
        df = pd.read_sql_query(query, cnx)
        return pn.widgets.Tabulator(df, show_index=False, sizing_mode='stretch_width', height=400)
    except Exception as e:
        return pn.pane.Alert(f'Erro na consulta: {str(e)}', alert_type='danger')

def on_inserir_user(w):
    """Insere um registro novo."""
    try:
        cursor = con.cursor()
        cursor.execute(
            "INSERT INTO pessoa(nome, cpf, nascimento, sexo, salario) VALUES (%s,%s,%s,%s,%s)",
            (w["nome"].value, w["cpf"].value, w["datanasc"].value, w["sexo"].value, w["salario"].value)
        )
        con.commit()
        cursor.close()
        return queryAll()
    except Exception as e:
        cursor.execute("ROLLBACK")
        cursor.close()
        return pn.pane.Alert(f'Não foi possível inserir: {str(e)}', alert_type='danger')

def on_atualizar_user(w):
    """Atualiza nome e salário pelo CPF."""
    try:
        cursor = con.cursor()
        cursor.execute(
            "UPDATE pessoa SET nome=%s, salario=%s WHERE cpf=%s",
            (
                w["nome"].value,
                w["salario"].value,
                w["cpf"].value
            )
        )
        con.commit()
        cursor.close()
        return queryAll()
    except Exception as e:
        cursor.execute("ROLLBACK")
        cursor.close()
        return pn.pane.Alert(f'Não foi possível atualizar: {str(e)}', alert_type='danger')

def on_excluir_user(w):
    """Exclui registro pelo CPF."""
    try:
        cursor = con.cursor()
        cursor.execute("DELETE FROM pessoa WHERE cpf=%s", (w["cpf"].value,))
        con.commit()
        cursor.close()
        return queryAll()
    except Exception as e:
        cursor.execute("ROLLBACK")
        cursor.close()
        return pn.pane.Alert(f'Não foi possível excluir: {str(e)}', alert_type='danger')


In [ ]:
def on_consultar_campaign(w):
    """Consulta pelo CPF ou retorna todos."""
    try:
        cpf = w["cpf"].value
        query = f"SELECT * FROM usuario WHERE ('{cpf}'='{flag}' OR cpf='{cpf}')"
        df = pd.read_sql_query(query, cnx)
        return pn.widgets.Tabulator(df, show_index=False, sizing_mode='stretch_width', height=400)
    except Exception as e:
        return pn.pane.Alert(f'Erro na consulta: {str(e)}', alert_type='danger')

def on_inserir_campaign(w):
    """Insere um registro novo."""
    try:
        cursor = con.cursor()
        cursor.execute(
            "INSERT INTO pessoa(nome, cpf, nascimento, sexo, salario) VALUES (%s,%s,%s,%s,%s)",
            (w["nome"].value, w["cpf"].value, w["datanasc"].value, w["sexo"].value, w["salario"].value)
        )
        con.commit()
        cursor.close()
        return queryAll()
    except Exception as e:
        cursor.execute("ROLLBACK")
        cursor.close()
        return pn.pane.Alert(f'Não foi possível inserir: {str(e)}', alert_type='danger')

def on_atualizar_campaign(w):
    """Atualiza nome e salário pelo CPF."""
    try:
        cursor = con.cursor()
        cursor.execute(
            "UPDATE pessoa SET nome=%s, salario=%s WHERE cpf=%s",
            (
                w["nome"].value,
                w["salario"].value,
                w["cpf"].value
            )
        )
        con.commit()
        cursor.close()
        return queryAll()
    except Exception as e:
        cursor.execute("ROLLBACK")
        cursor.close()
        return pn.pane.Alert(f'Não foi possível atualizar: {str(e)}', alert_type='danger')

def on_excluir_campaign(w):
    """Exclui registro pelo CPF."""
    try:
        cursor = con.cursor()
        cursor.execute("DELETE FROM pessoa WHERE cpf=%s", (w["cpf"].value,))
        con.commit()
        cursor.close()
        return queryAll()
    except Exception as e:
        cursor.execute("ROLLBACK")
        cursor.close()
        return pn.pane.Alert(f'Não foi possível excluir: {str(e)}', alert_type='danger')


In [ ]:
def on_consultar_cpoint(w):
    """Consulta pelo CPF ou retorna todos."""
    try:
        cpf = w["cpf"].value
        query = f"SELECT * FROM usuario WHERE ('{cpf}'='{flag}' OR cpf='{cpf}')"
        df = pd.read_sql_query(query, cnx)
        return pn.widgets.Tabulator(df, show_index=False, sizing_mode='stretch_width', height=400)
    except Exception as e:
        return pn.pane.Alert(f'Erro na consulta: {str(e)}', alert_type='danger')

def on_inserir_cpoint(w):
    """Insere um registro novo."""
    try:
        cursor = con.cursor()
        cursor.execute(
            "INSERT INTO pessoa(nome, cpf, nascimento, sexo, salario) VALUES (%s,%s,%s,%s,%s)",
            (w["nome"].value, w["cpf"].value, w["datanasc"].value, w["sexo"].value, w["salario"].value)
        )
        con.commit()
        cursor.close()
        return queryAll()
    except Exception as e:
        cursor.execute("ROLLBACK")
        cursor.close()
        return pn.pane.Alert(f'Não foi possível inserir: {str(e)}', alert_type='danger')

def on_atualizar_cpoint(w):
    """Atualiza nome e salário pelo CPF."""
    try:
        cursor = con.cursor()
        cursor.execute(
            "UPDATE pessoa SET nome=%s, salario=%s WHERE cpf=%s",
            (
                w["nome"].value,
                w["salario"].value,
                w["cpf"].value
            )
        )
        con.commit()
        cursor.close()
        return queryAll()
    except Exception as e:
        cursor.execute("ROLLBACK")
        cursor.close()
        return pn.pane.Alert(f'Não foi possível atualizar: {str(e)}', alert_type='danger')

def on_excluir_cpoint(w):
    """Exclui registro pelo CPF."""
    try:
        cursor = con.cursor()
        cursor.execute("DELETE FROM pessoa WHERE cpf=%s", (w["cpf"].value,))
        con.commit()
        cursor.close()
        return queryAll()
    except Exception as e:
        cursor.execute("ROLLBACK")
        cursor.close()
        return pn.pane.Alert(f'Não foi possível excluir: {str(e)}', alert_type='danger')


In [ ]:
def on_consultar_donation(w):
    """Consulta pelo CPF ou retorna todos."""
    try:
        cpf = w["cpf"].value
        query = f"SELECT * FROM usuario WHERE ('{cpf}'='{flag}' OR cpf='{cpf}')"
        df = pd.read_sql_query(query, cnx)
        return pn.widgets.Tabulator(df, show_index=False, sizing_mode='stretch_width', height=400)
    except Exception as e:
        return pn.pane.Alert(f'Erro na consulta: {str(e)}', alert_type='danger')

def on_inserir_donation(w):
    """Insere um registro novo."""
    try:
        cursor = con.cursor()
        cursor.execute(
            "INSERT INTO pessoa(nome, cpf, nascimento, sexo, salario) VALUES (%s,%s,%s,%s,%s)",
            (w["nome"].value, w["cpf"].value, w["datanasc"].value, w["sexo"].value, w["salario"].value)
        )
        con.commit()
        cursor.close()
        return queryAll()
    except Exception as e:
        cursor.execute("ROLLBACK")
        cursor.close()
        return pn.pane.Alert(f'Não foi possível inserir: {str(e)}', alert_type='danger')

def on_atualizar_donation(w):
    """Atualiza nome e salário pelo CPF."""
    try:
        cursor = con.cursor()
        cursor.execute(
            "UPDATE pessoa SET nome=%s, salario=%s WHERE cpf=%s",
            (
                w["nome"].value,
                w["salario"].value,
                w["cpf"].value
            )
        )
        con.commit()
        cursor.close()
        return queryAll()
    except Exception as e:
        cursor.execute("ROLLBACK")
        cursor.close()
        return pn.pane.Alert(f'Não foi possível atualizar: {str(e)}', alert_type='danger')

def on_excluir_donation(w):
    """Exclui registro pelo CPF."""
    try:
        cursor = con.cursor()
        cursor.execute("DELETE FROM pessoa WHERE cpf=%s", (w["cpf"].value,))
        con.commit()
        cursor.close()
        return queryAll()
    except Exception as e:
        cursor.execute("ROLLBACK")
        cursor.close()
        return pn.pane.Alert(f'Não foi possível excluir: {str(e)}', alert_type='danger')


In [ ]:
def table_creator(cons, ins, atu, exc, w):
    """
    Recebe booleans dos botões e executa a ação correspondente.
    """
    if cons: return on_consultar_user(w)
    if ins: return on_inserir(w)
    if atu: return on_atualizar(w)
    if exc: return on_excluir(w)


In [ ]:
# Cria uma ligação interativa (bind) entre os botões e a função que executa a ação correspondente,
# atualizando a tabela na interface sempre que algum botão for clicado.

# Conecta os botões à tabela interativa
form_widgets = create_form_widgets()

def create_interactive_table():
    return pn.bind(
        table_creator,
        buttonConsultar,
        buttonInserir,
        buttonAtualizar,
        buttonExcluir,
        form_widgets
    )


In [ ]:
# Tela de CRUD

# Monta o layout da interface com Panel:
# - Coluna esquerda com o título, os campos de entrada e os botões de ação
# - Coluna direita com a tabela interativa que mostra os dados do banco
# O método `.servable()` permite que essa interface seja exibida ao rodar o Panel server

def create_crud_view(title, widgets):
    """
    Cria a tela do CRUD.
    """
    crud_layout = pn.Row(
        pn.Column(
            pn.pane.Markdown(f"### 📋 CRUD {title}"),
            *[widgets[k] for k in widgets],
            pn.Row(buttonConsultar, buttonInserir, buttonAtualizar, buttonExcluir),
            pn.Spacer(height=10),
            create_btn_voltar(),
            sizing_mode='stretch_width',
            margin=(20, 20, 20, 20)
        ),
        pn.Column(create_interactive_table(), sizing_mode='stretch_both')
    )
    
    return crud_layout


In [ ]:
def go_to_user_crud():
    main_area.clear()
    main_area.append(create_crud_view("Usuário", form_widgets))

def go_to_donation_crud():
    main_area.clear()
    main_area.append(create_crud_view("Doação", get_donation_widgets()))

def go_to_campaign_crud():
    main_area.clear()
    main_area.append(create_crud_view("Campanha", get_campaign_widgets()))

def go_to_cpoint_crud():
    main_area.clear()
    main_area.append(create_crud_view("Ponto de Coleta", get_cpoint_widgets()))


In [ ]:
# Tela de Gráficos

def grafico_salario_por_sexo():
    df = pd.read_sql_query("SELECT sexo, AVG(salario) AS media_salario FROM pessoa GROUP BY sexo", cnx)
    fig, ax = plt.subplots()
    ax.bar(df["sexo"], df["media_salario"], color=['#1f77b4', '#ff7f0e', '#2ca02c'])
    ax.set_title("Salário Médio por Sexo")
    ax.set_ylabel("Salário Médio")
    return pn.pane.Matplotlib(fig, tight=True, sizing_mode='stretch_width')

def grafico_distribuicao_salario():
    df = pd.read_sql_query("SELECT salario FROM pessoa", cnx)
    fig, ax = plt.subplots()
    ax.hist(df["salario"], bins=10, color='#17becf')
    ax.set_title("Distribuição de Salários")
    ax.set_xlabel("Salário")
    ax.set_ylabel("Quantidade")
    return pn.pane.Matplotlib(fig, tight=True, sizing_mode='stretch_width')


In [ ]:
def create_graficos_view():
    graficos_layout = pn.Column(
        pn.pane.Markdown("### 📊 Análise de Funcionários"),
        grafico_salario_por_sexo,
        grafico_distribuicao_salario,
        create_btn_voltar(),
        sizing_mode='stretch_width',
        margin=(20,20,20,20)
    )
    
    return graficos_layout

In [ ]:
def go_to_graficos():
    main_area.clear()
    main_area.append(create_graficos_view())

In [ ]:
# Conecta os botões da Home às funções de navegação
btn_go_user_crud.on_click(lambda event: go_to_user_crud())
btn_go_donation_crud.on_click(lambda event: go_to_donation_crud())
btn_go_campaign_crud.on_click(lambda event: go_to_campaign_crud())
btn_go_cpoint_crud.on_click(lambda event: go_to_cpoint_crud())
btn_go_graficos.on_click(lambda event: go_to_graficos())

In [ ]:
# Inicializa com a tela Home
go_to_home()

In [ ]:
# App principal, com título e área dinâmica
pn.Column(
    pn.pane.Markdown("# 🏢 Sistema de Funcionários"),
    main_area
).servable()